In [ ]:
import numpy as np
import nibabel as nib
from dipy.core.gradients import gradient_table
from dipy.reconst.dti import TensorModel

def normalize(x):
    return (x - np.mean(x)) / np.std(x)

Load data

In [ ]:
def load_subject_data(data_path):
    data = nib.load(fr'{data_path}\Diffusion\data.nii.gz').get_fdata()
    bvals = np.loadtxt(fr'{data_path}\Diffusion\bvals')
    bvecs = np.loadtxt(fr'{data_path}\Diffusion\bvecs')
    mask = nib.load(fr'{data_path}\Diffusion\nodif_brain_mask.nii.gz').get_fdata()
    T1w = nib.load(fr'{data_path}\T1w_acpc_dc_restore_1.25.nii.gz').get_fdata()
    return data, bvals, bvecs, mask, T1w

In [ ]:
subject_ID = '100206'
data_path = fr'..\diffusion_data\{subject_ID}\T1w'
data, bvals, bvecs, mask, T1w = load_subject_data(data_path)

In [ ]:
print(f"Data shape: {data.shape}")
print(f"Number of b-values: {len(bvals)}")
print(f"Mask shape: {mask.shape}")
print(f"T1w shape: {T1w.shape}")

Prepare data

In [ ]:
optimized_directions = np.array([
    [0.416, 0, 0.910],
    [-0.416, 0, 0.910],
    [0.910, -0.416, 0],
    [0.910, 0.416, 0],
    [0, 0.910, 0.416],
    [0, 0.910, -0.416]
])

In [ ]:
def select_and_transform_dwis(data, bvals, bvecs):
    b0_val = np.min(bvals)
    b0_indices = np.where(bvals == b0_val)[0]
    b0_image = np.mean(data[:,:,:,b0_indices], axis=3)

    gtab = gradient_table(bvals, bvecs)
    highest_bval = np.max(bvals)
    high_b_indices = np.where(bvals == highest_bval)[0]
    
    angles = np.abs(np.dot(bvecs.T[high_b_indices], optimized_directions.T))
     

select_and_transform_dwis(data, bvals, bvecs)